# Heart Disease Prediction
Mengyang He and Milo Yen-Goossens

In [45]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [46]:
data = pd.read_csv('./cardio_train.csv',sep=";")
df = pd.DataFrame(data)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  int64  
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  int64  
 8   gluc         70000 non-null  int64  
 9   smoke        70000 non-null  int64  
 10  alco         70000 non-null  int64  
 11  active       70000 non-null  int64  
 12  cardio       70000 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 6.9 MB


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1


In [47]:
df.drop(columns=['id'],inplace=True)
df

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
69995,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,22431,1,163,72.0,135,80,1,2,0,0,0,1


In [48]:
# check if the dataset has null or missing value
df.isnull().sum()

age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
dtype: int64

In [49]:
# remove egregious (i.e. impossible) systolic and diastolic blood pressure values

from scipy import stats

threshold = 3

ap_hi_outliers = df[df['ap_hi'] > 250]
df = df.drop(ap_hi_outliers.index)

ap_lo_outliers = df[df['ap_lo'] > 150]
df = df.drop(ap_lo_outliers.index)

In [50]:
# add bmi feature
bmi = df['weight'].div(df['height'].multiply(df['height'])) * 100
df['bmi'] = bmi

In [51]:
# make normalized dataframe
df_normalized = df.copy()
for var in ['age', 'height', 'weight', 'ap_hi', 'ap_lo']:
  print(var + " mean, avg: " + str(df[var].mean()) + ", " + str(df[var].std()))
  df_normalized[var] = df[var] - df[var].mean()
  df_normalized[var] = df_normalized[var].div(df[var].std())

df_normalized

age mean, avg: 19463.59660795825, 2468.38480682597
height mean, avg: 164.3598898311227, 8.204045411568867
weight mean, avg: 74.11611944625642, 14.328253541828976
ap_hi mean, avg: 126.29723853011524, 17.861674927889364
ap_lo mean, avg: 81.30319634703196, 9.761402032165053


,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,bmi
0,-0.433724,2,0.443697,-0.845610,-0.912414,-0.133505,1,1,0,0,1,0,0.219671
1,0.309678,1,-1.018996,0.759610,0.767160,0.890938,3,1,0,0,1,1,0.349277
2,-0.245746,1,0.078024,-0.706026,0.207302,-1.157948,3,1,0,0,0,1,0.235078
3,-0.745668,2,0.565588,0.550233,1.327018,1.915381,1,1,0,0,1,1,0.287105
4,-0.806032,1,-1.018996,-1.264363,-1.472272,-2.182391,1,1,0,0,0,0,0.230112
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,-0.090584,2,0.443697,0.131480,-0.352556,-0.133505,1,1,1,0,1,0,0.269274
69996,1.271035,1,-0.775214,3.621089,0.767160,0.890938,2,2,0,0,1,1,0.504727
69997,-0.161076,2,2.272063,2.155453,3.006592,0.890938,3,1,0,1,0,1,0.313536
69998,1.202164,1,-0.165758,-0.147689,0.487231,-0.133505,1,2,0,0,0,1,0.270993


In [53]:
# logistic regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

X_log = df_normalized.drop(columns = ["cardio"])
Y_log = df_normalized["cardio"]

X_log_train , X_log_test , Y_log_train , Y_log_test = train_test_split(X_log,Y_log,test_size=0.3,random_state=0)
acc_log = 0
# best_C = float('-inf')
# C_vals = 10**np.arange(-8.,20.)
# for C in C_vals:
logreg = LogisticRegression()
logreg.fit(X_log_train, Y_log_train)
acc_log = max(acc_log, round(logreg.score(X_log_train, Y_log_train) * 100, 5))
# best_C = max(best_C, C)
print("Accureacy of Logistic Regression training set:" ,acc_log)

acc_test_log = round(logreg.score(X_log_test, Y_log_test) * 100, 5)
print("Accureacy of Logistic Regression validation set:" ,acc_test_log)


Accureacy of Logistic Regression training set: 72.65216
Accureacy of Logistic Regression validation set: 72.47294


In [54]:
# Correlation of Logisctic Regression
coeff_df = pd.DataFrame(X_log_train.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])
coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
4,ap_lo,0.830074
6,gluc,0.510609
0,gender,0.357705
5,cholesterol,0.194872
3,ap_hi,0.171589
2,weight,-0.033474
1,height,-0.034375
8,alco,-0.094873
7,smoke,-0.123713
10,bmi,-0.211337


In [75]:
log_pre = list(logreg.predict(X_log_test))
log_true_y = list(Y_log_test)
print(len(log_pre))
print(len(log_true_y))

20696
20696


In [84]:
def accuracy_metrics(y, predictions):
  pos = 0
  neg = 0
  truePos = 0
  trueNeg = 0
  falsePos = 0
  falseNeg = 0

  for i in range(len(y)):
    if predictions[i] == 1:
      if y[i] == 1:
        pos += 1
        truePos += 1
      else:
        neg += 1
        falsePos += 1
    else:
      if y[i] == 1:
        pos += 1
        falseNeg += 1
      else:
        neg += 1
        trueNeg += 1

  return (truePos / pos, trueNeg / neg, falsePos / neg, falseNeg / pos)



print(accuracy_metrics(log_pre, log_true_y))
print(accuracy_metrics(log_true_y, log_pre))

(0.7539883375508857, 0.7018178685276126, 0.29818213147238737, 0.24601166244911432)
(0.6644366879968974, 0.7846272394528992, 0.21537276054710075, 0.3355633120031026)


In [55]:
# ANN
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import optimizers

def build_ann(optimizer='adam'):
    # Initializing the ANN
    ann = Sequential()
    
    # Adding the input layer and the first hidden layer of the ANN with dropout
    ann.add(Dense(units=32, kernel_initializer='glorot_uniform', activation='relu', input_shape=(len(X_ann_train.columns),)))
    
    # Add other layers, it is not necessary to pass the shape because there is a layer before
    ann.add(Dense(units=64, kernel_initializer='glorot_uniform', activation='relu'))
    ann.add(Dropout(rate=0.5))
    ann.add(Dense(units=64, kernel_initializer='glorot_uniform', activation='relu'))
    ann.add(Dropout(rate=0.5))
    
    # Adding the output layer
    ann.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))
    
    # Compiling the ANN
    ann.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return ann

#train, test, target, target_test = train_test_split(df_normalized, df_nomalized["cardio"], test_size=0.2, random_state=0)

X_ann = df_normalized.drop(columns = ["cardio"])
Y_ann = df_normalized["cardio"]
X_ann_train , X_ann_test , Y_ann_train , Y_ann_test = train_test_split(X_ann,Y_ann,test_size=0.3,random_state=0)

opt = optimizers.Adam(lr=0.001)
ann = build_ann(opt)
# Training the ANN
history = ann.fit(X_ann_train, Y_ann_train, batch_size=16, epochs=100, validation_data=(X_ann_test, Y_ann_test))

# Predicting the Train set results
ann_prediction = ann.predict(X_ann_train)
ann_prediction = (ann_prediction > 0.5)*1 # convert probabilities to binary output

Epoch 1/100
3019/3019 [==============================] - 2s 482us/step - loss: 0.5785 - accuracy: 0.7146 - val_loss: 0.5490 - val_accuracy: 0.7296
Epoch 2/100
3019/3019 [==============================] - 1s 444us/step - loss: 0.5634 - accuracy: 0.7270 - val_loss: 0.5457 - val_accuracy: 0.7320
Epoch 3/100
3019/3019 [==============================] - 1s 440us/step - loss: 0.5591 - accuracy: 0.7272 - val_loss: 0.5477 - val_accuracy: 0.7308
Epoch 4/100
3019/3019 [==============================] - 1s 446us/step - loss: 0.5576 - accuracy: 0.7284 - val_loss: 0.5474 - val_accuracy: 0.7311
Epoch 5/100
3019/3019 [==============================] - 1s 443us/step - loss: 0.5566 - accuracy: 0.7299 - val_loss: 0.5432 - val_accuracy: 0.7320
Epoch 6/100
3019/3019 [==============================] - 1s 450us/step - loss: 0.5559 - accuracy: 0.7289 - val_loss: 0.5408 - val_accuracy: 0.7353
Epoch 7/100
3019/3019 [==============================] - 1s 479us/step - loss: 0.5556 - accuracy: 0.7309 - val_loss: 0

3019/3019 [==============================] - 1s 438us/step - loss: 0.5473 - accuracy: 0.7341 - val_loss: 0.5427 - val_accuracy: 0.7342
Epoch 57/100
3019/3019 [==============================] - 1s 442us/step - loss: 0.5487 - accuracy: 0.7322 - val_loss: 0.5434 - val_accuracy: 0.7355
Epoch 58/100
3019/3019 [==============================] - 1s 468us/step - loss: 0.5483 - accuracy: 0.7331 - val_loss: 0.5458 - val_accuracy: 0.7326
Epoch 59/100
3019/3019 [==============================] - 1s 452us/step - loss: 0.5482 - accuracy: 0.7339 - val_loss: 0.5419 - val_accuracy: 0.7350
Epoch 60/100
3019/3019 [==============================] - 1s 473us/step - loss: 0.5466 - accuracy: 0.7339 - val_loss: 0.5404 - val_accuracy: 0.7344
Epoch 61/100
3019/3019 [==============================] - 1s 476us/step - loss: 0.5493 - accuracy: 0.7338 - val_loss: 0.5410 - val_accuracy: 0.7366
Epoch 62/100
3019/3019 [==============================] - 1s 442us/step - loss: 0.5479 - accuracy: 0.7336 - val_loss: 0.5438 

In [89]:
# Training set results
ann_prediction = ann.predict(X_ann_train)
ann_prediction = (ann_prediction > 0.5)*1 # convert probabilities to binary output

# Compute error between predicted data and true response and display it in confusion matrix
acc_ann = round(metrics.accuracy_score(Y_ann_train, ann_prediction) * 100, 2)
print("Accureacy of ANN training set:", acc_ann)

1510/1510 [==============================] - 0s 254us/step
Accureacy of ANN training set: 73.63


In [90]:
# Predicting the Test set results
ann_prediction_test = ann.predict(X_ann_test)
ann_prediction_test = (ann_prediction_test > 0.5)*1 # convert probabilities to binary output

# Compute error between predicted data and true response and display it in confusion matrix
acc_test_ann = round(metrics.accuracy_score(Y_ann_test, ann_prediction_test) * 100, 5)
print("Accureacy of ANN validation set:",acc_test_ann)

647/647 [==============================] - 0s 268us/step
Accureacy of ANN validation set: 73.24604


In [94]:
ann_pre = list(ann.predict(X_ann_test))
ann_true_y = list(Y_ann_test)
print(len(ann_pre))
print(len(ann_true_y))
print(ann_pre)
print(ann_true_y)

647/647 [==============================] - 0s 264us/step
20696
20696
[array([0.472229], dtype=float32), array([0.77177376], dtype=float32), array([0.37579238], dtype=float32), array([0.92212737], dtype=float32), array([0.7256078], dtype=float32), array([0.18578984], dtype=float32), array([0.27823943], dtype=float32), array([0.39645043], dtype=float32), array([0.20390227], dtype=float32), array([0.3426362], dtype=float32), array([0.05356744], dtype=float32), array([0.31280404], dtype=float32), array([0.9119081], dtype=float32), array([0.38591677], dtype=float32), array([0.25290272], dtype=float32), array([0.48911542], dtype=float32), array([0.09766906], dtype=float32), array([0.36351037], dtype=float32), array([0.8620833], dtype=float32), array([0.8004597], dtype=float32), array([0.23649982], dtype=float32), array([0.40351185], dtype=float32), array([0.8937032], dtype=float32), array([0.5419727], dtype=float32), array([0.30368152], dtype=float32), array([0.36340517], dtype=float32), arr

In [58]:
from keras.utils import plot_model
from tensorflow.keras.utils import plot_model

keras.utils.plot_model(
    ann,
    to_file="model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=200,
    show_layer_activations=True,
    show_trainable=True,
)
#plot_model(ann, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [59]:
print(ann.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                416       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 6753 (26.38 KB)
Trainable params: 6753 (26

In [77]:
# SVM
from sklearn.svm import SVC, LinearSVC
from sklearn import svm
#train, test, target, target_test = train_test_split(df_normalized, df_nomalized["cardio"], test_size=0.2, random_state=0)

X_svm = df_normalized.drop(columns = ["cardio"])
Y_svm = df_normalized["cardio"]
X_svm_train , X_svm_test , Y_svm_train , Y_svm_test = train_test_split(X_svm,Y_svm,test_size=0.3,random_state=0)

clf = svm.SVC(C = 2, kernel='rbf')
clf.fit(X_svm_train, Y_svm_train)
acc_svc = round(clf.score(X_svm_train, Y_svm_train) * 100, 5)
print("Accureacy of SVM training set:" ,acc_svc)

acc_test_svc = round(clf.score(X_svm_test, Y_svm_test) * 100, 5)
print("Accureacy of SVM validation set:" ,acc_test_svc)

Accureacy of SVM training set: 73.74764
Accureacy of SVM validation set: 73.6954


In [78]:
svm_pre = list(clf.predict(X_svm_test))
svm_true_y = list(Y_svm_test)
print(len(svm_pre))
print(len(svm_true_y))

20696
20696


In [83]:
print(accuracy_metrics(svm_pre, svm_true_y))
print(accuracy_metrics(svm_true_y, svm_pre))

(0.7717027449230083, 0.7104141810124425, 0.28958581898755753, 0.22829725507699175)
(0.6705448904401784, 0.8029281448661144, 0.19707185513388556, 0.3294551095598216)
